In [1]:
# 3 main ways to reduce the variance of MC simulation


# [1.] display the normal way without reduction:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

# construct the fuction:
def withoutReductionSimulate(scheme,parameters):
    
    # before start the simulation, initialize the random:
    np.random.seed(1000)
    
   #--------------------------------------------# 

    # total time:
    T=parameters['setup']['T']
    
    # number of steps
    numSteps = parameters['setup']['numSteps'] 
    
    # number of simulated paths
    numPaths = parameters['setup']['numPaths']  
    dt = parameters['setup']['dt']

    # model parameters
    S_0 = parameters['model']['S0']  # initial value
    
    sigma = parameters['model']['sigma']  # initial value
    
    rf = parameters['model']['rf']  # initial value    
    
    #-------------------------------------------#
    
    # simulation preparation:
    S=np.zeros((numSteps+1,numPaths),d=float)
    
    # setup the identical S_0
    S[0,:]=np.log(S_0)
    
    for i in range(numPaths):
        for i in range(1,numSteps+1):
            #need to cosntruct a standard normal variable to simulate the 
            #the process,which is z:
            Z=np.random.normal(0,1,1)
            
            
            if scheme == 'Euler':
                S[t_step,i] = S[t_step-1,i] + (rf-sigma**2/2)*dt + sigma*np.sqrt(dt)*Zs 
            elif scheme == 'Milstein':    
                # Euler and Milstein have the same discretization scheme for Log(S) due to dsigma(t,X)/dX =0
                S[t_step,i] = S[t_step-1,i] + (rf-sigma**2/2)*dt + sigma*np.sqrt(dt)*Zs                                
    
    return np.exp(S)

In [ ]:
# [1.] display the first way to reduction-->control variance:

def firstReduction_control(scheme,parameters):
    
    # before start the simulation, initialize the random:
    np.random.seed(1000)
    
   #--------------------------------------------# 
    
    # time setup
    T = parameters['setup']['T']  # total time/maturity
    numSteps = parameters['setup']['numSteps']  # number of steps
    numPaths = parameters['setup']['numPaths']  # number of simulated paths
    dt = parameters['setup']['dt']

    # model parameters
    S_0 = parameters['model']['S0']  # initial value
    sigma = parameters['model']['sigma']  # initial value
    rf = parameters['model']['rf']  # initial value 
    
    
    # simulation preparation:
    S=np.zeros((numSteps+1,numPaths),dtype=float)


In [2]:
if __name__=="__main__": # main function
    
    #==================================================
    #    Environment Setup
    #================================================== 
    
    # underlying info
    S0 = 100
    sigma = 0.20    
    rf = 0.05       
    
    # option info    
    K = 100
    T = 1     # maturity
    Optype = 1 # 1: call -1: put 
    
    # discrete setup
    N = 200  # steps
    dt = T/N
    M = 1
    
    # info structure
    parameters = {'model':{'S0':S0, 'sigma':sigma, 'rf':rf},                 
                  'asset':{'K':K, 'optype':Optype},
                  'option':{'K':0, 'optype':0, 'T':0},   # save all option info
                  'setup':{'T':T, 'numSteps':N, 'dt': T/N, 'numPaths':M},                
                  'strategy':{'n_c':0, 'n_s':0, 'n_b':0, 'k':0} # new added parameters  
                 }
    
     
        
def Black_ScholesPrice(parameters):

    S0 = parameters['model']['S0']
    sigma = parameters['model']['sigma']    
    rf = parameters['model']['rf']  
    
    K = parameters['option']['K']    
    T = parameters['option']['T']
    option_type = parameters['option']['optype'] 
    
    DF = np.exp(-rf*T)
    
    d1 = (np.log(S0/K)+(rf+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    #print('S0=%f, K=%f, sigm=%f, T= %f, d1=%f vs d2=%f'%(S0, K,sigma, T, d1,d2))

    if option_type == 1:  # call
        option_price = S0 * norm.cdf(d1) - K * DF * norm.cdf(d2)
    elif option_type == -1: # put
        option_price = K * DF * norm.cdf(-d2) - S0 * norm.cdf(-d1) 
    
    return option_price


def Black_ScholesPrice_Paras(St, K, sigma, T, rf, option_type):

    DF = np.exp(-rf*T)
    
    d1 = (np.log(St/K)+(rf+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 1:  # call
        option_price = St * norm.cdf(d1) - K * DF * norm.cdf(d2)
    elif option_type == -1: # put
        option_price = K * DF * norm.cdf(-d2) - St * norm.cdf(-d1) 
    
    return option_price


def Black_Scholes_Delta(parameters):

    # based on the lecture, delta = N(d1)
    
    S0 = parameters['model']['S0']
    sigma = parameters['model']['sigma']    
    rf = parameters['model']['rf']
    
    K = parameters['option']['K'] 
    T = parameters['option']['T']  
    option_type = parameters['option']['optype'] 
    
    d1 = (np.log(S0/K)+(rf+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    
    if option_type == 1:  # call    
        bs_delta = norm.cdf(d1)
    else:    
        bs_delta = norm.cdf(d1) - 1    
    
    return bs_delta 

def Black_Scholes_Gamma(parameters):
    
    # based on the lecture, gamma = phi(d1)/S_t/(sigma*sqrt(T))
    
    S0 = parameters['model']['S0']
    sigma = parameters['model']['sigma']    
    rf = parameters['model']['rf']
    
    K = parameters['option']['K'] 
    T = parameters['option']['T']    
    
    d1 = (np.log(S0/K)+(rf+0.5*sigma**2)*T)/(sigma*np.sqrt(T))
    
    bs_gamma = norm.pdf(d1)/S0/(sigma*np.sqrt(T))
    
    return bs_gamma 

def EulerMilsteinMCStock(scheme, parameters):
    
    # is to show all the possible stock paths not to reduce the extreme value 
    
    np.random.seed(1000)

    # time setup
    T = parameters['setup']['T']  # total time/maturity
    numSteps = parameters['setup']['numSteps']  # number of steps
    numPaths = parameters['setup']['numPaths']  # number of simulated paths
    dt = parameters['setup']['dt']

    # model parameters
    S_0 = parameters['model']['S0']  # initial value
    sigma = parameters['model']['sigma']  # initial value
    rf = parameters['model']['rf']  # initial value    

    # simulation    
    S = np.zeros((numSteps + 1, numPaths),dtype=float)
    S[0,:] = np.log(S_0)
  
    ################         simluations for asset price S              ########
    for i in range(numPaths):
        for t_step in range(1, numSteps+1):
            # the 2 stochastic drivers for variance V and asset price S and correlated
            Zs = np.random.normal(0, 1, 1)                        
            
            if scheme == 'Euler':
                S[t_step,i] = S[t_step-1,i] + (rf-sigma**2/2)*dt + sigma*np.sqrt(dt)*Zs 
            elif scheme == 'Milstein':    
                # Euler and Milstein have the same discretization scheme for Log(S) due to dsigma(t,X)/dX =0
                S[t_step,i] = S[t_step-1,i] + (rf-sigma**2/2)*dt + sigma*np.sqrt(dt)*Zs                                
            
    return np.exp(S)


#----------------strategy------------------#
def DeltaGammaNeutralHedging(Sim_S,parameters):

    # for delta-neutral hedging, k=0
    # for delta-gamma-neutral hedging, k = -n_c*gamma_1/gamma_2
    # strategy
    n_c = parameters['strategy']['n_c'] 
    n_s = parameters['strategy']['n_s'] 
    n_b = parameters['strategy']['n_b'] 
    k = parameters['strategy']['k']   # k == 0:   delta-neutral and otherwise delta-gamma-neutral
    
    # time setup
    T = parameters['setup']['T']  # total time/maturity
    numSteps = parameters['setup']['numSteps']  # number of steps
    numPaths = parameters['setup']['numPaths']  # number of simulated paths
    dt = parameters['setup']['dt']
    
    # model
    sigma = parameters['model']['sigma']  # initial value
    rf = parameters['model']['rf']   
    
    # Option_1
    K = parameters['asset']['K']  # Option_1 
    option_type_1 = parameters['asset']['optype'] 
    
    # Option_2
    K_2 = parameters['option']['K']  # Option_2
    T_2 = parameters['option']['T']   
    option_type_2 = parameters['option']['optype'] 
    
    ################         replication errors              ########
    PI = np.zeros((numSteps + 1, numPaths),dtype=float)
    PI[0,:] = 0
    
    for i in range(numPaths):
        for t_step in range(1, numSteps+1):
            S_t_i = Sim_S[t_step,i]# S(1,0)
            T_new = T - dt * t_step  # update the maturity of option_1
            # MOVE formward, reduce the maturity time 
            T_2_new = T_2 - dt * t_step  #update the maturity of option_2
            
            if T_new > 0:     # before T
                price_1_t_i = Black_ScholesPrice_Paras(S_t_i, K, sigma, T_new, rf, option_type_1)  # option_1 is not expired                
            elif T_new == 0:   # at T
                price_1_t_i = np.maximum(0, (S_t_i- K) if option_type_1 == 1 else (K-S_t_i)) # option_2 is not expired 
            
            if k != 0: # delta-gamma-neutral hedging
                price_2_t_i = Black_ScholesPrice_Paras(S_t_i, K_2, sigma, T_2_new, rf, option_type_2)   # option_2 is not expired due to T1<T2 
            else:       # delta-neutral hedging
                price_2_t_i = 0
            
            #print('price_1_t_i=%f vs price_2_t_i=%f'%(price_1_t_i, price_2_t_i))
    
            # work out replication errors
            # NOTE:
            # since the portfolio is not rebalanced dynamically over time, the time-t value of PI is not ensured to be zero. 
            # the self-financing requirement requires the zero value of PI at each time t, suggesting a dynamic trading strategy
            # in this sense, we only talk about static trading strategy in this demo, but the idea should be very clear
            PI[t_step,i] = n_c*price_1_t_i + n_s*S_t_i + k*price_2_t_i + n_b*(1+rf*dt)  
            # k=0 when the neutural delta method
            
    return PI        

    #==================================================
    #    Task 2 -- delta-gamma-neutral strategy (static)
    #    Key ideas: 1) the strategy is established at t=0
    #               2) hold it until maturity T wihout rebalancing
    #               3) replication errors are recored during [0,T]
    #================================================== 
    # setup   
    
    #option_2  at t =0 
    parameters_2 = parameters  
    parameters_2['option']['K'] = K
    parameters_2['option']['optype'] = Optype  # call
    parameters_2['option']['T'] = T + 1 # with longer maturity  
    # option 2 的 maturity 更长
    
    BS_price_2 = Black_ScholesPrice(parameters_2)    
    delta_2 = Black_Scholes_Delta(parameters_2)
    gamma_2 = Black_Scholes_Gamma(parameters_2)
    
    parameters_2['strategy']['n_c'] = -1 # -1: a short position in Option 1
    parameters_2['strategy']['k'] = -parameters_2['strategy']['n_c']*gamma_1/gamma_2
    parameters_2['strategy']['n_s'] = -parameters_2['strategy']['n_c']*delta_1 - parameters_2['strategy']['k']* delta_2
    parameters_2['strategy']['n_b'] = -(parameters_2['strategy']['n_c']*BS_price_1 
                                        + parameters_2['strategy']['n_s']* parameters_2['model']['S0']
                                        + parameters_2['strategy']['k']* BS_price_2
                                       )     
    print('parameter_2=\n')
    print(parameters_2)

    print('option_1=%f vs option_2 = %f'%(BS_price_1, BS_price_2))
    print('delta_1=%f vs delta_2=%f'%(delta_1, delta_2))
    print('gamma_1=%f vs gamma_2=%f'%(gamma_1, gamma_2))
    
    #perform delta-gamma-neutral hedging 
    PI_DGN = DeltaGammaNeutralHedging(Sim_S,parameters_2)
    
    PI_DGN_DF = pd.DataFrame(PI_DGN) 
    PI_DGN_DF.to_excel(r"PI_DGN_paths.xlsx")